<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2021/examples/blob/main/example-kafka/Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read https://towardsdatascience.com/kafka-python-explained-in-10-lines-of-code-800e3e07dad1

In [ ]:
#zookeeper is packaged in a debian package that you can install
#kazoo is the python library used to work with zookeeper
!apt-get install zookeeperd
!pip install kazoo

In [2]:
#install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
#get kafka
!wget  https://downloads.apache.org/kafka/2.7.0/kafka_2.13-2.7.0.tgz
!tar -zxf kafka_2.13-2.7.0.tgz
!sudo mv kafka_2.13-2.7.0 /usr/local/kafka
!sudo mkdir -p /tmp/kafka-logs

In [ ]:
!/usr/share/zookeeper/bin/zkServer.sh start

In [5]:
!/usr/local/kafka/bin/kafka-server-start.sh -daemon /usr/local/kafka/config/server.properties

In [6]:
!/usr/local/kafka/bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic bigdata

Created topic bigdata.


In [7]:
#verify the topics
!/usr/local/kafka/bin/kafka-topics.sh --zookeeper localhost:2181  --describe --topic bigdata

Topic: bigdata	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: bigdata	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [ ]:
#produce a message 
!/usr/local/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic bigdata

>

In [ ]:
#consume messages
!/usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic bigdata --from-beginning

Processed a total of 21 messages


In [8]:
#let us now try to write a simple python program to produce and consume messages.
!pip3 install kafka-python

     |████████████████████████████████| 256kB 4.3MB/s 


In [9]:
#kafka Producer
from time import sleep
from json import dumps
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
for e in range(10):
    data = {'number' : e}
    producer.send('bigdata', value=data)
    print("sent {}".format(data))
    sleep(2)


sent {'number': 0}
sent {'number': 1}
sent {'number': 2}
sent {'number': 3}
sent {'number': 4}
sent {'number': 5}
sent {'number': 6}
sent {'number': 7}
sent {'number': 8}
sent {'number': 9}


In [11]:
#so here is a basic consumer as well
from kafka import KafkaConsumer
from json import loads
consumer = KafkaConsumer(
    'bigdata',
     bootstrap_servers=['localhost:9092'],
     consumer_timeout_ms=1000,#important for stopping if there is no data
     auto_offset_reset='earliest',
     value_deserializer=lambda x: loads(x.decode('utf-8')))
for message in consumer:
    message = message.value   
    print('got {}'.format(message))

got {'number': 0}
got {'number': 1}
got {'number': 2}
got {'number': 3}
got {'number': 4}
got {'number': 5}
got {'number': 6}
got {'number': 7}
got {'number': 8}
got {'number': 9}
